# Selenium WEB SCRAPING ASSIGNMENT-2 

In [3]:
#lets import all the required libraries
import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException

## Question 1

In [346]:
def question1(url):
    
    #activate chrome browser
    driver= webdriver.Chrome("chromedriver.exe")
    
    
    driver.get(url)  #making request
    
    df_naukri = pd.DataFrame({})   #creating a empty dataframe
    
    
    time.sleep(5)
    #finding the element for Designation search bar
    designation_search = driver.find_element_by_id('qsb-keyword-sugg')
    #write on search bar
    #asking for user input
    #Designation = input('Enter the Designation you want to search : ')
    designation_search.send_keys("Data Analyst") #user input as data analyst
    
    time.sleep(2)
    #finding the element for Location search bar
    loaction_search = driver.find_element_by_id('qsb-location-sugg')
    #write on search bar
    #Location = input('Enter the Location you want to search :')
    loaction_search.send_keys('Banglore')  #uesr input as banglore
    
    time.sleep(2)
    #do click using xpath function
    search_button = driver.find_element_by_xpath("//button[@class='btn']")
    search_button.click()   #clicking the  search button
    
    time.sleep(5)  #sleep for 4 seconds
    
    #so extarct all the tags having job-titles
    title_tags = driver.find_elements_by_xpath("//a[@class= 'title fw500 ellipsis']")
    job_title_list = []   #creating a empty list
    for i in title_tags[:10]:  #we only need 10 elements
        job_title_list.append(i.text)   #appending it to the list
    
    #lets get all the company name
    company_tags = driver.find_elements_by_xpath("//a[@class = 'subTitle ellipsis fleft']")
    company_list = []
    for i in company_tags[:10]:
        company_list.append(i.text)
        
    #lets extract the experice
    #some data is missin in span tags, lets use parent tag li
    experience_tags = driver.find_elements_by_xpath("//li[@class = 'fleft grey-text br2 placeHolderLi experience']//span")
    experience_list = []
    for i in experience_tags[:10]:
        experience_list.append(i.text)
    
    #lets extract the loaction
    #some data is missin in span tags, lets use parent tag li
    location_tags = driver.find_elements_by_xpath("//li[@class = 'fleft grey-text br2 placeHolderLi location']//span")
    location_list = []
    for i in location_tags[:10]:
        location_list.append(i.text)

    
    
    df_naukri['Job Title'] = job_title_list  #Creating a column for the particular list
    df_naukri['Company'] = company_list
    df_naukri['Experience'] = experience_list
    df_naukri['Location'] = location_list
    
 
    return df_naukri  #returning the Dataframe as result


In [347]:
#calling the function to display the dataframe as result
question1('https://www.naukri.com/')

,Job Title,Company,Experience,Location
0,Data Scientist / Data Analyst -Business Analyst,Inflexion Analytix Private Limited,0-3 Yrs,"Mumbai, Hyderabad/Secunderabad, Pune, Gurgaon/..."
1,Data Analyst,GlaxoSmithKline Pharmaceuticals Limited,0-2 Yrs,Bangalore/Bengaluru
2,Hiring Data Analysts For E commerce Platform |...,Allegis Services India Pvt. Ltd.,0-5 Yrs,Bangalore/Bengaluru
3,Data Analyst,Shell India Markets Private Limited,5-8 Yrs,Bangalore/Bengaluru
4,Data Analyst,Applied Materials,7-10 Yrs,Bangalore/Bengaluru
5,Hiring For Data Analyst/ MIS Reporting Analyst...,PHARMACEUTICAL RESEARCH ASSOCIATES INDIA Pvt Ltd,2-4 Yrs,Bangalore/Bengaluru
6,DA - Urgent Opening For Data Analyst BFSI Doma...,Tata Consultancy Services Ltd.,4-9 Yrs,"Kolkata, Hyderabad/Secunderabad, Pune, Ahmedab..."
7,Data Analyst - Informatica MDM,Shell India Markets Private Limited,6-9 Yrs,Bangalore/Bengaluru
8,Data Analyst,Shell India Markets Private Limited,5-8 Yrs,Bangalore/Bengaluru
9,Assistant Vice President - MIS & Reporting ( B...,INTERTRUSTVITEOS CORPORATE AND FUND SERVICES P...,12-18 Yrs,"Mumbai, Bangalore/Bengaluru"


## Question 2

Q2: Write a python program to scrape data for “Data Scientist” Job position in
“Bangalore” location. You have to scrape the job-title, job-location,
company_name, full job-description. You have to scrape first 10 jobs data.

In [348]:
def question2(url):
    #activate chrome browser
    driver= webdriver.Chrome("chromedriver.exe")
    driver.get(url)  #making request
    
    time.sleep(5)
    #finding the element for Designation search bar
    designation_search = driver.find_element_by_id('qsb-keyword-sugg')
    #write on search bar
    #asking for user input
    #Designation = input('Enter the Designation you want to search : ')
    designation_search.send_keys("Data Scientist") #user input as data analyst

    time.sleep(2)
    #finding the element for Location search bar
    loaction_search = driver.find_element_by_id('qsb-location-sugg')
    #write on search bar
    #Location = input('Enter the Location you want to search :')
    loaction_search.send_keys('Banglore')  #uesr input as banglore

    time.sleep(2)
    #do click using xpath function
    search_button = driver.find_element_by_xpath("//button[@class='btn']")
    search_button.click()   #clicking the  search button
    
    
    job_links = []  
    time.sleep(6)
    j_link =driver.find_elements_by_xpath("//a[@class ='title fw500 ellipsis']") #getting all links 
    for i in j_link:
        job_links.append(i.get_attribute('href')) #appendig the links after filtering data scientist and banglore
        
    j_title = [] #creating empty list
    c_name = []
    experience_list = []
    j_description = []

    
    #there are some non naukri pages after clicking , for example IBM
    #IBM page are having diffrent tags and classes
    #so to skip them i used expection handling
    
    naukri_page_count = 0  #after reaching 10, we will break out of loop
    not_naukri_page = 0   #just for cross checking
    for j in job_links:  #looping all links
        try:
            driver.get(j)
            time.sleep(4)
            naukri_page = driver.find_element_by_id('smjltButton')  #finding some elemet on naukri page
        except NoSuchElementException:
            not_naukri_page += 1    #if elemet is not found, then it is not a naukri page
        else:
            #job title
            job_title = driver.find_element_by_xpath("//h1[@class= 'jd-header-title']")
            j_title.append(job_title.text)   
            
            #copamny name
            company_name  = driver.find_element_by_xpath("//div[@class= 'jd-header-comp-name']/a")
            c_name.append(company_name.text)
            
            #experience required
            experience = driver.find_element_by_xpath("//div[@class= 'exp']/span")
            experience_list.append(experience.text)
            
            #full description
            full_description = driver.find_element_by_xpath("//section[@class= 'job-desc']")
            j_description.append(full_description.text)

            #increasing count
            naukri_page_count += 1

            if naukri_page_count == 10:  #if information is retrived for 10 pages, break the loop
                break
    
    naukri_job_desc_df = pd.DataFrame({})    #creating a empty dataframe
    
    naukri_job_desc_df['Job Title'] = j_title   #Creating a column for the particular list
    naukri_job_desc_df['Comapany'] =c_name
    naukri_job_desc_df['Experience required'] =experience_list
    naukri_job_desc_df['Full decription'] =j_description
    
    #theses are some non naukri pages count
    print('we found {} not naukri page while scraping, so skipped those pages'.format(not_naukri_page))
    
    return naukri_job_desc_df   #returning the Dataframe as result

In [349]:
#calling the function to display the dataframe as result
question2('https://www.naukri.com/')

we found 6 not naukri page while scraping, so skipped those pages


,Job Title,Comapany,Experience required,Full decription
0,Data Scientist / Data Analyst -Business Analyst,Inflexion Analytix Private Limited,0 - 3 years,Job description\nJob Role : Data Scientist/Dat...
1,Senior Data Scientist | CES IT LTD | CMMI Level 5,CES Ltd.,2 - 7 years,Job description\nRoles and Responsibilities\n\...
2,Senior Data Scientist | CES IT LTD | CMMI Level 5,CES Ltd.,2 - 7 years,Job description\nRoles and Responsibilities\n\...
3,Senior Data Scientist,Vijaya Management Services,5 - 10 years,Job description\n.\nRoles and Responsibilities...
4,"Senior Data Scientist, Modeling",Nielsen,3 - 7 years,Job description\nWe wont say we can predict th...
5,Senior Data Scientist - Credit risk,Scienaptic Systems,5 - 10 years,Job description\n\nResponsibilities and duties...
6,SDE Lead Data Scientist-L3,Huawei Technologies India Pvt Ltd,5 - 8 years,Job description\nBusiness & Team overview:\nFo...
7,Computational Design Lead Data Scientist-L3,Huawei Technologies India Pvt Ltd,5 - 8 years,Job description\nBusiness & Team overview:\nFo...
8,Lead Data Scientist,Intel Technology India Pvt Ltd,6 - 10 years,Job description\n\n\nWe are seeking an outstan...
9,Senior Data Scientist - Chatbot & NLP,Gojek Tech,3 - 7 years,Job description\nWhat You Will Do\nWork with D...


## Question 3

In [352]:
def question3(url):
    
    #activate chrome browser
    driver= webdriver.Chrome("chromedriver.exe")
    
    
    driver.get(url)  #making request
    
    df_naukri = pd.DataFrame({})   #creating a empty dataframe
    
    time.sleep(3)
    #finding the element for Designation search bar
    designation_search = driver.find_element_by_id('qsb-keyword-sugg')
    #write on search bar
    #asking for user input
    #Designation = input('Enter the Designation you want to search : ')
    designation_search.send_keys("Data Scientist") #user input as data analyst
    
    
    #do click using xpath function
    search_button = driver.find_element_by_xpath("//button[@class='btn']")
    search_button.click()   #clicking the  search button
    
    time.sleep(4)  #sleep for 4 seconds
    
    #adding Filter for Loaction Delhi/NCR
    loc_filter = driver.find_element_by_xpath('html/body/div[1]/div[3]/div[2]/section[1]/div[2]/div[2]/div[2]/div[2]/label/i')
    loc_filter.click()
    time.sleep(4)
    
    #adding filter for salary (3-6 lakhs)
    time.sleep(4)
    salary_filter = driver.find_element_by_xpath('html/body/div[1]/div[3]/div[2]/section[1]/div[2]/div[3]/div[2]/div[2]/label/i')
    salary_filter.click()  
    time.sleep(4)
    
    
    #so extarct all the tags having job-titles
    title_tags = driver.find_elements_by_xpath("//a[@class= 'title fw500 ellipsis']")
    job_title_list = []   #creating a empty list
    for i in title_tags[:10]:  #we only need 10 elements
        job_title_list.append(i.text)   #appending it to the list
    
    #lets get all the company name
    company_tags = driver.find_elements_by_xpath("//a[@class = 'subTitle ellipsis fleft']")
    company_list = []
    for i in company_tags[:10]:
        company_list.append(i.text)
        
    #lets extract the experice
    #some data is missin in span tags, lets use parent tag li
    experience_tags = driver.find_elements_by_xpath("//li[@class = 'fleft grey-text br2 placeHolderLi experience']//span")
    experience_list = []
    for i in experience_tags[:10]:
        experience_list.append(i.text)
    
    #lets extract the loaction
    #some data is missin in span tags, lets use parent tag li
    location_tags = driver.find_elements_by_xpath("//li[@class = 'fleft grey-text br2 placeHolderLi location']//span")
    location_list = []
    for i in location_tags[:10]:
        location_list.append(i.text)

    
    
    df_naukri['Job Title'] = job_title_list  #Creating a column for the particular list
    df_naukri['Company'] = company_list
    df_naukri['Experience'] = experience_list
    df_naukri['Location'] = location_list
    
 
    return df_naukri  #returning the Dataframe as result


In [353]:
#calling the function to display the dataframe as result
question3('https://www.naukri.com/')

,Job Title,Company,Experience,Location
0,Data Scientist / Data Analyst -Business Analyst,Inflexion Analytix Private Limited,0-3 Yrs,"Mumbai, Hyderabad/Secunderabad, Pune, Gurgaon/..."
1,Data Scientist,IBM India Pvt. Limited,4-9 Yrs,"Noida, Hyderabad/Secunderabad, Bangalore/Benga..."
2,Excellent opportunity For Data Scientist,NEC CORPORATION INDIA PRIVATE LIMITED,3-7 Yrs,"Noida, Bangalore/Bengaluru"
3,Data Scientist - Noida/ B'lore,NEC CORPORATION INDIA PRIVATE LIMITED,3-8 Yrs,"Noida, Bangalore/Bengaluru"
4,Data Scientist,Cloudstrats Technologies Private Limited,5-8 Yrs,"Bangalore/Bengaluru, Delhi / NCR, Mumbai (All ..."
5,Data Scientist,Country Veggie,1-3 Yrs,"Bharuch, Jaipur, Bhopal, Mumbai, Jhansi, Nagpu..."
6,Data Scientist,BlackBuck,3-7 Yrs,"Gurgaon/Gurugram, Bangalore/Bengaluru"
7,Data Scientist,Mobikwik,3-5 Yrs,"New Delhi, Gurgaon/Gurugram, Delhi / NCR"
8,DATA Scientist – Gurgaon (Exp 3-6 years),CRESCENDO GLOBAL LEADERSHIP HIRING INDIA PRIVA...,3-6 Yrs,"Gurgaon/Gurugram, Delhi / NCR"
9,DATA Scientist – Gurgaon (Exp 3-6 years),CRESCENDO GLOBAL LEADERSHIP HIRING INDIA PRIVA...,3-6 Yrs,"Gurgaon/Gurugram, Delhi / NCR"


## Question 4

In [354]:
def question4(url):
    #activate chrome browser
    driver= webdriver.Chrome("chromedriver.exe")
    #opening the homepage of amazon.in
    driver.get(url)
    
    time.sleep(4)
    #We have sign in to this page first
    
    #it was asking signin evrytime, so i made it automated
    #you can remove this part if not asking in your system
    sign_in = driver.find_element_by_xpath("//button[@class= 'd-none d-lg-block p-0 LockedHomeHeaderStyles__signInButton']")
    time.sleep(4)
    sign_in.click()  #sigining in
    
    
    #providing the email
    time.sleep(4)
    Email  = driver.find_element_by_id("userEmail")
    Email.clear()
    Email.send_keys('ullas1ga15cs167@gmail.com')
    
    #providing the password
    password = driver.find_element_by_id('userPassword')
    password.clear()
    password.send_keys('Ullas#1234')
    
    #enter after providing both email and password
    time.sleep(2)
    password.send_keys(Keys.ENTER)
    
    #you can comment out the above steps if directly sigined in in your system
    
    #job title input
    time.sleep(5)
    job_title = driver.find_element_by_id('sc.keyword')
    job_title.send_keys('Data scientist') #sending the key as Data scientist
    
    
    #finding job job location input
    time.sleep(4)
    job_location = driver.find_element_by_id('sc.location')
    time.sleep(4)
    job_location.send_keys(Keys.CONTROL + "a")  #selecting if anything there in input box
    time.sleep(2)
    job_location.send_keys(Keys.DELETE)  #clearing the input
    job_location.send_keys('Noida')  #send the key as noida
    
    time.sleep(2)
    #search button
    search_button = driver.find_element_by_xpath("//button[@class = 'gd-ui-button ml-std col-auto SearchStyles__newSearchButton css-iixdfr']")
    search_button.click()  #clicking on search button
    
    
    time.sleep(8)  #sometimes it will be very slow
    
    #company name
    company_name = []
    c_names = driver.find_elements_by_xpath("//div[@class='d-flex justify-content-between align-items-start']/a/span")
    for i in c_names[:10]: #we need only ten elememts
        company_name.append(i.text)  #appending it to the list
        
    #job titles
    job_titles = []
    j_titles = driver.find_elements_by_xpath("//a[@class='jobLink css-1rd3saf eigr9kq2']/span")
    for j in j_titles[:10]:
        job_titles.append(j.text)
    
    
    #how many days ago the job was posted
    job_posted = []
    j_posted = driver.find_elements_by_xpath("//div[@class='d-flex align-items-end pl-std css-mi55ob']")
    for k in j_posted[:10]:
        job_posted.append(k.text)
    
    job_glassdoor_df = pd.DataFrame({})   #creating a empty dataframe
    job_glassdoor_df['Company'] = company_name   #Creating a column for the particular list
    job_glassdoor_df['Job Title'] =job_titles
    job_glassdoor_df['job_posted'] = job_posted
    
    return job_glassdoor_df    #returning the Dataframe as result

In [355]:
#calling the function to display the dataframe as result
question4(' https://www.glassdoor.co.in/index.htm')

,Company,Job Title,job_posted
0,Biz2Credit Inc,Data Scientist,30d+
1,Ericsson,Data Scientist,8d
2,Techlive,Data Scientist,30d+
3,Salasar New Age Technologies,Data Scientist,30d+
4,Salasar New Age Technologies,Data Scientist Intern,30d+
5,UnitedHealth Group,"Data Scientist - Gurgaon, HR",24h
6,CRMNEXT,Data Scientist - Computer Vision,7d
7,WishFin,Data Scientist,30d+
8,CRMNEXT,Data Scientist – Voice & NLP,7d
9,SearchUrCollege,Data Scientist,30d+


## Question 5

In [356]:
def question5(url):
    #activate chrome browser
    driver= webdriver.Chrome("chromedriver.exe")
    #opening the homepage of amazon.in
    driver.get(url)
    
    
    time.sleep(6) #sleep 6 seconds
    job_title = driver.find_element_by_id('KeywordSearch')  #finding title input
    job_title.send_keys('Data Scientist')   #sending input
    
    time.sleep(4)
    job_location = driver.find_element_by_id('LocationSearch')  #finding location input
    time.sleep(4)
    job_location.send_keys(Keys.CONTROL + "a")  #clearing location flied
    job_location.send_keys(Keys.DELETE)
    job_location.send_keys('Noida')  #sending input
    
    time.sleep(2)
    search_button = driver.find_element_by_id("HeroSearchButton")  #seach button with id
    search_button.click()  #clicking search button
     
    
    time.sleep(6)
    company_names = []   #creating a empty list
    c_names = driver.find_elements_by_xpath("//div[@class='d-flex']/div[2]/p[2]")
    for i in c_names[:10]:  #since we need only 10
        company_names.append(i.text)  #appending it to the list
    
    average_salary = []
    #creating xpath associated with the output required
    a_salary = driver.find_elements_by_xpath("//div[@class='col-2 d-none d-md-flex flex-row justify-content-end']/strong")
    for j in a_salary[:10]:
        average_salary.append(j.text)
    
    minimum_salary = []
    min_salary = driver.find_elements_by_xpath("//div[@class = 'common__RangeBarStyle__values d-flex justify-content-between ']/span[1]")
    for k in min_salary[:10]:
        minimum_salary.append(k.text)
   
    maximum_salary = []
    max_salary = driver.find_elements_by_xpath("//div[@class = 'common__RangeBarStyle__values d-flex justify-content-between ']/span[2]")
    for l in max_salary[:10]:
        maximum_salary.append(l.text)
        
    salary_df = pd.DataFrame({})   #creating a empty dataframe
    
    salary_df["Company"] = company_names  #Creating a column for the particular list
    salary_df["Average Salary"] =average_salary
    salary_df['Minimum Salary'] = minimum_salary
    salary_df['Maximum Salary'] = maximum_salary
    
    return salary_df  #returning the Dataframe as result

In [357]:
#calling the function to display the dataframe as result
question5('https://www.glassdoor.co.in/Salaries/index.htm')

,Company,Average Salary,Minimum Salary,Maximum Salary
0,Tata Consultancy Services,"₹ 6,11,228",₹343K,"₹1,095K"
1,Accenture,"₹ 11,46,533",₹577K,"₹2,213K"
2,IBM,"₹ 8,97,795",₹586K,"₹2,730K"
3,Ericsson-Worldwide,"₹ 7,38,057",₹355K,"₹1,613K"
4,Delhivery,"₹ 12,39,781",₹450K,"₹11,622K"
5,UnitedHealth Group,"₹ 13,36,142","₹1,069K","₹1,520K"
6,Valiance Solutions,"₹ 8,15,192",₹502K,"₹1,465K"
7,ZS Associates,"₹ 11,35,221",₹202K,"₹1,809K"
8,EXL Service,"₹ 11,44,243",₹575K,"₹1,520K"
9,Optum Global Solutions,"₹ 14,13,288","₹1,014K","₹2,149K"


## Question 6

In [358]:
def question6(url):
    
    #activate chrome browser
    driver= webdriver.Chrome("chromedriver.exe")
    
    #opening the flipcart.com
    driver.get(url)
    
    
    #we can skip the login option by closing the popup
    #because i am getting loh=gin option everytime
    #if you are getting login popup, please comment out the below three lines
    time.sleep(4)
    skip_login = driver.find_element_by_xpath("//button[@class ='_2KpZ6l _2doB4z']")
    skip_login.click()
    
    time.sleep(4)
    #finding the element for product search bar
    product_search = driver.find_element_by_xpath("//div[@class = '_3OO5Xc']/input")
    #write on search bar
    product_search.send_keys("sunglasses")
    
    
    #finding the search button with xpath
    search =driver.find_element_by_xpath("//button[@class = 'L0Z3Pu']") 
    search.click()  #clicking the button
    
    SG_names = []
    SG_price = []
    SG_desc = []
    SG_discount = []
    
    x = 0
    while len(SG_names) < 100:
        
    
        time.sleep(5)
        #finding the name of sunglasses

        name = driver.find_elements_by_xpath("//div[@class= '_2WkVRV']")   #finding the Xpath associated with particular class
        for i in name:
            SG_names.append(i.text)   #appending it to the list

        #finding the name of sunglasses
        price = driver.find_elements_by_xpath("//div[@class= '_30jeq3']")
        for j in price:
            SG_price.append(j.text)  

        #finding the product description of sunglasses
        desc = driver.find_elements_by_xpath("//div[@class= '_2B099V']/a[1]")
        for k in desc:
            SG_desc.append(k.text)

        #finding the discount of sunglasses
        
        discount = driver.find_elements_by_xpath("//div[@class= '_3Ay6Sb']/span")
        for l in discount:
            SG_discount.append(l.text)
        
        #navigating to the next page
        if x == 0:
            Next_page = driver.find_element_by_xpath("//a[@class= '_1LKTO3']/span")  #xpath will get changed form 2nd page
            Next_page.click()   #clicking next
        else:
            Next_page = driver.find_element_by_xpath("//nav[@class= 'yFHi8N']/a[12]/span")
            Next_page.click()
            
        x += 1   # counter
    
    sunglasses_df = pd.DataFrame({})   #creating a empty dataframe
    
    sunglasses_df['Brand'] = SG_names[:100]  #since we need only 100 product details
    sunglasses_df['Price'] = SG_price[:100]
    sunglasses_df['Discount'] = SG_discount[:100]
    sunglasses_df['Description'] = SG_desc[:100]

    return sunglasses_df   #returning the Dataframe as result

In [359]:
#calling the function to display the dataframe as result
question6('https://www.flipkart.com/')

,Brand,Price,Discount,Description
0,hipe,₹219,78% off,Mirrored Wayfarer Sunglasses (55)
1,hipe,₹210,85% off,"Mirrored, UV Protection, Gradient Round Sungla..."
2,PIRASO,₹250,84% off,UV Protection Aviator Sunglasses (54)
3,Fastrack,₹758,15% off,UV Protection Wayfarer Sunglasses (Free Size)
4,Fastrack,₹630,21% off,UV Protection Rectangular Sunglasses (Free Size)
...,...,...,...,...
95,hipe,₹210,83% off,"UV Protection, Night Vision, Riding Glasses, G..."
96,hipe,₹199,80% off,UV Protection Wayfarer Sunglasses (Free Size)
97,GANSTA,₹224,77% off,"UV Protection, Mirrored Wayfarer Sunglasses (53)"
98,Silver Kartz,₹255,78% off,UV Protection Aviator Sunglasses (62)


## Question 7

In [361]:
def question7(url):
    
    #activate chrome browser
    driver= webdriver.Chrome("chromedriver.exe")
    
    #opening the flipcart.com
    driver.get(url)
    
    
    #all review button
    time.sleep(5)
    All_review = driver.find_element_by_xpath("//div[@class = '_3UAT2v _16PBlm']/span")
    All_review.click()
    
    
    #finding the ratings of all iphones
    iphone_rating = []     #returning the list
    iphone_summary = []
    iphone_review = []
    x = 0
    while len(iphone_rating) < 100:  #stopping the while loop at 100 

        time.sleep(4)   #sleep 4 seconds after every loop
         
        #Rating provided by the user
        #finding the tag associated with particular class
        i_rating = driver.find_elements_by_xpath("//div[@class = 'col _2wzgFH K0kLPL']/div[1]/div")
        for i in i_rating:
            iphone_rating.append(i.text)  #appending it to the list
        
        #summary provided by the user
        i_summary = driver.find_elements_by_xpath("//div[@class = 'col _2wzgFH K0kLPL']/div[1]/p")
        for j in i_summary:
            iphone_summary.append(j.text)
            
        #review provided by the user
        i_review = driver.find_elements_by_xpath("//div[@class = 't-ZTKy']/div/div")
        for k in i_review:
            iphone_review.append(k.text)

        if x == 0:
            #next button in first page
            time.sleep(1)
            next_button_p1 = driver.find_element_by_xpath("//a[@class = '_1LKTO3']/span")  #xpath will get changed form 2nd page
            next_button_p1.click()
        else:
            #next button from second page
            next_button = driver.find_element_by_xpath("//nav[@class = 'yFHi8N']/a[12]/span")
            next_button.click()
            x += 1

        

    
    iphone_df = pd.DataFrame({})   #creating a empty dataframe
    
    iphone_df['Rating'] = iphone_rating[:100]  #since we need only 100 product details
    iphone_df['Summary'] = iphone_summary[:100]
    iphone_df['Review'] = iphone_review[:100]

    return iphone_df   #returning the Dataframe as result


In [362]:
#calling the function to display the dataframe as result
question7('https://www.flipkart.com/apple-iphone-11-black-64-gb-includesearpods-poweradapter/p/itm0f37c2240b217?pid=MOBFKCTSVZAXUHGR&lid=LSTMOBFKCTSVZAXUHGREPBFGI&marketplace')

#run try it again if you get the error.

,Rating,Summary,Review
0,5,Brilliant,The Best Phone for the Money\n\nThe iPhone 11 ...
1,5,Perfect product!,Amazing phone with great cameras and better ba...
2,4,Good choice,So far it’s been an AMAZING experience coming ...
3,5,Great product,Amazing Powerful and Durable Gadget.\n\nI’m am...
4,5,Worth every penny,Previously I was using one plus 3t it was a gr...
...,...,...,...
95,5,Terrific,Really worth of money. i just love it. It is t...
96,5,Highly recommended,It's my first time to use iOS phone and I am l...
97,5,Wonderful,This is my first ever I phone. Before this I w...
98,5,Classy product,Superb Product !!!\nA big and worthy upgrade f...


## Question 8

In [363]:
def question8(url):
    
    #activate chrome browser
    driver= webdriver.Chrome("chromedriver.exe")
    #opening the flipcart.com
    driver.get('https://www.flipkart.com/')
    
    time.sleep(3)
    skip_login = driver.find_element_by_xpath("//button[@class ='_2KpZ6l _2doB4z']")
    skip_login.click()
    
    #finding the element for product search bar
    product_search = driver.find_element_by_xpath("//div[@class = '_3OO5Xc']/input")
    #write on search bar
    product_search.send_keys("sneakers")
    
    #finding the search button with xpath
    search =driver.find_element_by_xpath("//button[@class = 'L0Z3Pu']") 
    search.click()  #clicking the button


    #finding the tag associated with particular class
    shoe_brand = []
    shoe_desc = []
    shoe_price = []
    shoe_discount = []
    x =0

    while len(shoe_brand) < 100:

        time.sleep(5)

        brand = driver.find_elements_by_xpath("//div[@class = '_2WkVRV']")
        for i in brand:
            shoe_brand.append(i.text)  #appending it to the list

        desc = driver.find_elements_by_xpath("//div[@class = '_2B099V']/a[1]")
        for j in desc:
            shoe_desc.append(j.text)

        price = driver.find_elements_by_xpath("//div[@class = '_30jeq3']")
        for k in price:
            shoe_price.append(k.text)

        discount = driver.find_elements_by_xpath("//div[@class = '_3Ay6Sb']/span")
        for l in discount:
            shoe_discount.append(l.text)

        if x == 0:
            #next button in first page
            next_button_p1 = driver.find_element_by_xpath("//a[@class = '_1LKTO3']/span")  #xpath will get changed form 2nd page
            next_button_p1.click()
        else:
            #next button from second page
            next_button = driver.find_element_by_xpath("//nav[@class = 'yFHi8N']/a[12]/span")
            next_button.click()

        x += 1


    sneakers_df = pd.DataFrame({})   #creating a empty dataframe

    sneakers_df['Brand'] = shoe_brand[:100]  #since we need only 100 product details
    sneakers_df['Description'] = shoe_desc[:100]
    sneakers_df['Price'] = shoe_price[:100]
    sneakers_df['Discount'] = shoe_discount[:100]

    return sneakers_df  #returning the Dataframe as result
    


In [364]:
#calling the function to display the dataframe as result
question8('https://www.flipkart.com/')

,Brand,Description,Price,Discount
0,well feet,sneaker for mens and boys Sneakers For Men,₹429,28% off
1,Bucik,Multicolor Casual Shoes Mesh for Men Sneakers ...,₹599,70% off
2,Robbie jones,Casual Sneakers Shoes For Men Sneakers For Men,₹399,60% off
3,aadi,Sneakers For Men,₹298,70% off
4,Numenzo,Sneakers For Men,₹398,60% off
...,...,...,...,...
95,JUBENTA,Latest Design Sneakers For Men,₹549,60% off
96,ROCKFIELD,Sneakers For Men,₹399,82% off
97,Adiso,"Casual sneakers,outdoors,dancingshoes Sneakers...",₹399,48% off
98,India hub,Fashionable casual sneakers shoes Sneakers For...,₹449,50% off


## Question 9

In [365]:
def question9(url):
    #activate chrome browser
    driver= webdriver.Chrome("chromedriver.exe")
    #opening the myntra.com
    driver.get(url)
    
    #finding price filter range 6k - 13 k
    time.sleep(8)
    price_filter = driver.find_element_by_xpath("//ul[@class = 'price-list']/li[2]/label/div")
    price_filter.click()
    
    #finding color filter black
    time.sleep(6)
    color_filter = driver.find_element_by_xpath("//li[@class = 'colour-listItem'][1]/label/div")
    color_filter.click()
    
    s_brand_list = []   #creating a empty list
    s_desc_list = []
    s_price_list = []
    
    while len(s_brand_list) < 100:  #breaking while loop when 100 is reached
        
        time.sleep(5) #sleep for 5 seconds after every loop
        
        #Find the brand name
        s_brand = driver.find_elements_by_xpath("//div[@class = 'product-productMetaInfo']/h3")
        for i in s_brand:
            s_brand_list.append(i.text)
        len(s_brand_list)

        #Finding the description

        s_desc = driver.find_elements_by_xpath("//div[@class = 'product-productMetaInfo']/h4[1]")
        for j in s_desc:
            s_desc_list.append(j.text)
        len(s_desc_list)

        #Finding the price

        s_price = driver.find_elements_by_xpath("//div[@class = 'product-productMetaInfo']/div[1]/span[1]")
        for k in s_price:
            price = k.text[:9].replace('R','')  #some div tags have only one and some have two so have to do this
            a = 'R'+price
            s_price_list.append(a)
        len(s_price_list)

        #next button 
        next_button = driver.find_element_by_xpath("//li[@class = 'pagination-next']/a")
        next_button.click()
        
    myntra_shoe_df = pd.DataFrame({})   #creating a empty dataframe
    
    myntra_shoe_df['Brand'] = s_brand_list[:100]   #Creating a column for the particular list since we need only 100
    myntra_shoe_df['Price'] = s_price_list[:100]
    myntra_shoe_df['Description'] = s_desc_list[:100]
    
    return myntra_shoe_df   #returning the Dataframe as result
    

In [366]:
#calling the function to display the dataframe as result
question9('https://www.myntra.com/shoes')

,Brand,Price,Description
0,Nike,Rs. 10846,Men React Infinity Running
1,Nike,Rs. 11495,AIR ZOOM PEGASUS Running Shoes
2,Nike,Rs. 12495,Men JORDAN DELTA Basketball
3,PUMA Motorsport,Rs. 7999,Unisex Mercedes Running Shoes
4,Puma,Rs. 9999,Unisex RS-FAST TECH Sneakers
...,...,...,...
95,ADIDAS,Rs. 12749,Men Ultraboost DNA CC_1 Shoes
96,Nike,Rs. 8396,Men REACT PHANTOM FLYKNIT2 Run
97,Nike,Rs. 9371,Men REACT MILER PRM Running
98,Bugatti,Rs. 7999,Men Slip-On Sneakers


## Qusetion 10

In [43]:
def Question10(url):
    #activate chrome browser
    driver= webdriver.Chrome("chromedriver.exe")
    #opening the amazon.com
    driver.get(url)
    time.sleep(3)
    
    
    #finding the element for product search bar
    Laptop_search = driver.find_element_by_xpath("//div[@class='nav-search-field ']/input")
    #write on search bar
    Laptop_search.send_keys("Laptop")
    
    search_key = driver.find_element_by_id('nav-search-submit-button')
    search_key.click()
    
    
    #filter
    time.sleep(4)
    i7 = driver.find_element_by_xpath("//li[@id = 'p_n_feature_thirteen_browse-bin/12598163031']/span/a")
    i7.click()
    #we can't choose two processors i7 and i9 , because the page allows only one
    
    time.sleep(4)
    #lets create the list of first 10 links of laaptp after filter
    links_list = []
    links = driver.find_elements_by_xpath("//a[@class = 'a-link-normal a-text-normal']")
    for i in links[:10]:
        links_list.append(i.get_attribute('href')) #appendinf links

        
        
    name_list = []  #creating empty lists
    price_list = []
    rating_list = []
    for k in links_list:
        driver.get(k)  #getting lists one by one
        time.sleep(5)

        #name
        name = driver.find_element_by_xpath("//h1[@id = 'title']")    #getting name
        name_list.append(name.text)

        #price
        #price is not given for out of stock items or brand new items, so using exception handling
        try:
            price = driver.find_element_by_xpath("//span[@id = 'priceblock_ourprice']")
        except NoSuchElementException:
            price_list.append("Out of stcok, Price not available")
        else:
            price_list.append(price.text)



        #ratings
        #For some new laptops rating are not given, so lets use exception handlings
        try:
            review_link = driver.find_element_by_xpath("//span[@id = 'acrCustomerReviewText']")
        except NoSuchElementException:
            rating_list.append("Review Not Given")
        else:
            review_link.click()
            time.sleep(3)
            rating = driver.find_element_by_xpath("//span[@class =  'a-size-base a-nowrap']/span")
            rating_list.append(rating.text)

    
    
    laptop_df = pd.DataFrame({})    #creating a empty dataframe
    laptop_df['Name'] = name_list  #we need ony 10
    laptop_df['Price'] = price_list
    laptop_df['Rating'] = rating_list
    
    return laptop_df    #returning the Dataframe as result

In [42]:
#calling the function to display the dataframe as result
Question10('https://www.amazon.in/')

#i am not able retrieve rataings from this webpage

,Name,Price,Rating
0,(Renewed) Dell Latitude E6420 14 Inch Laptop (...,"₹ 34,000.00",Review Not Given
1,(Renewed) Lenovo T430 14-inch Laptop (Core i7/...,"₹ 53,499.00",Review Not Given
2,(Renewed) Lenovo T430 14-inch Laptop (Core i7/...,"₹ 37,499.00",Review Not Given
3,(Renewed) Lenovo T430 14-inch Laptop (Core i7/...,"₹ 37,999.00",Review Not Given
4,(Renewed) Lenovo Intel 4th Gen Core i7-4980HQ ...,"₹ 46,290.00",3 out of 5
5,Lenovo IdeaPad Gaming 3 10th Gen Intel Core i7...,"₹ 84,200.00",4.3 out of 5
6,"HP 14 Thin & Light 14"" (35.56cms) FHD Laptop (...","Out of stcok, Price not available",4.7 out of 5
7,"Asus ROG Zephyrus S Ultra Slim Gaming Laptop, ...","₹ 3,43,099.00",5 out of 5
8,(Renewed) Dell Intel 6th Gen Core i7-6820HQ 15...,"₹ 61,799.00",Review Not Given
9,(Renewed) Dell Latitude E7470 14-inch Laptop (...,"₹ 65,299.00",Review Not Given
